# Библиотеки для линейн программирования
- SciPy (scipy.optimize.linprog);
- CVXPY; 
- PuLP.

In [1]:
import numpy as np

# Примеры из юнита

## 1. Пример № 1. SciPy (scipy.optimize.linprog)

У нас есть 6 товаров с заданными ценами на них и заданной массой.  
Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.  
Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [2]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

- Сформулируем задачу линейного программирования. Максимизируем произведение стоимости на количество, учитывая, что произведение веса на искомое количество товаров должно укладываться во вместимость сумки:
$$
\max \sum v_i x_i
$$
$$
\sum w_i x_i \le C
$$
- векторно-матричная форма должна формулироваться в следующем виде:
$$
\min c^T x
$$
$$
Ax \le b
$$
- __Получается, что в наших обозначениях мы имеем следующее:__
$$
c=-x,  A = w^T,  b = (C)

In [ ]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива, так как изначально это вектор и размерность (6,), а нам нужна матрица
b = np.array([C]) #конвертируем вместимость в массив
A.ndim

2

In [15]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

__ВЫВОД:__ Получаем искомое значение функции —52  (в выводе значение с минусом, но мы меняем знак, возвращаясь к задаче максимизации).  
x = (0, 0, 0, 7.5, 0) .  
Таким образом, мы взяли только самую дорогую, четвёртую вещь. Она одна весит 2 кг, а если взять её 7.5 раз, то получится как раз 15 кг.
___

## 2. Пример № 2. CVXPY  
Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

> SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку CVXPY

In [29]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [30]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [ ]:
import cvxpy

# Cоздадим переменную-массив x. Укажем его размерность, а также условие, что все числа в массиве должны быть целыми
x = cvxpy.Variable(shape=n, integer=True)

In [ ]:
#Далее зададим ограничения, используя матричное умножение:
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

1

In [ ]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])
problem.solve()
#В результате получаем бесконечность. Это совершенно нереалистично.

In [ ]:
# в таком случае будем рассматривать только положительные значения x>=0. т.е добавим ограничение

x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

print(problem.solve())
print(x.value)

-49.0
[-0. -0. -0.  7. -0.  0.]


__ВЫВОД__: Здесь мы уже получаем 49, и берём только 4 товар в количестве семи штук. Можно увидеть, что результат, в целом, очень близок к первому, когда мы использовали библиотеку SciPy — различие лишь в добавлении целочисленности. Значит, у нас получилось решить задачу, когда мы добавили недостающее условие
___

In [43]:
#А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём x типа boolean x=0 или x=1

x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

print(problem.solve())
print(x.value)

-17.0
[1. 0. 0. 1. 0. 1.]


__ВЫВОД__: Получим стоимость, равную 17, взяв первый, четвёртый и шестой товары.
___

## Пример № 3. PuLP
В нашей каршеринговой компании две модели автомобилей: модель A и модель B. Автомобиль A даёт прибыль в размере 20 тысяч в месяц, а автомобиль B — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

- Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа A, и 5 дней — для каждого автомобиля типа B.
- Заводу требуется 3 дня, чтобы изготовить модель A, и 6 дней, чтобы изготовить модель B.
- Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль A, и 7 дней — автомобиль B.
- Каждый специалист может работать суммарно 30 дней.

- Целевая функция будет выглядеть следующим образом:  
$$
20000A + 45000B
$$
- Также запишем ограничения:
$$
{A, B} \le 0
$$
$$
4A + 5B \le 30
$$
$$
3A + 6B \le 30
$$
$$
2A + 7B\le 30
$$
> Заметьте, что здесь мы снова пишем обычные неравенства, а не условия в матричном виде. Дело в том, что для данной библиотеки так «удобнее», так как она принимает все условия в «первичном» виде.

In [94]:
from pulp import LpMaximize, LpProblem, LpVariable, LpInteger, value, LpMinimize
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\pulp\pulp.py:1489: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


__ВЫВОД__: Выходит, что необходимо произвести 1 автомобиль типа  и 4 автомобиля типа . Тогда суммарный чистый доход будет равен 200 тысячам.
___

# Самостоятельная работа

Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:  

![axis в DataFrame](MATHML_md6_6_1_1.png)  

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом.


In [95]:
problem = LpProblem('Transportation', LpMinimize)
w1s1 = LpVariable('w1s1', lowBound=0)
w1s2 = LpVariable('w1s2', lowBound=0)
w1s3 = LpVariable('w1s3', lowBound=0)
w2s1 = LpVariable('w2s1', lowBound=0)
w2s2 = LpVariable('w2s2', lowBound=0)
w2s3 = LpVariable('w2s3', lowBound=0)
#целевая функция
problem += 2*w1s1 + 5*w1s2 + 3*w1s3 + 7*w2s1 + 7*w2s2 + 6*w2s3

#огранич по складам
problem += w1s1 + w1s2 + w1s3 == 180
problem += w2s1 + w2s2 + w2s3 == 220

#ограничения по ТЦ
problem += w1s1 + w2s1 == 110
problem += w1s2 + w2s2 == 150
problem += w1s3 + w2s3 == 140

problem.solve()

print(value(problem.objective))

1900.0


In [ ]:
problem = LpProblem('Work', LpMinimize)
w1t1 = LpVariable('w1s1', lowBound=0)
w1s2 = LpVariable('w1s2', lowBound=0)
w1s3 = LpVariable('w1s3', lowBound=0)
w1s4 = LpVariable('w1s2', lowBound=0)
w1s5 = LpVariable('w1s3', lowBound=0)

w2s1 = LpVariable('w2s1', lowBound=0)
w2s2 = LpVariable('w2s2', lowBound=0)
w2s3 = LpVariable('w2s3', lowBound=0)
w2s4 = LpVariable('w2s2', lowBound=0)
w2s5 = LpVariable('w2s3', lowBound=0)

w3s1 = LpVariable('w3s1', lowBound=0)
w3s2 = LpVariable('w3s2', lowBound=0)
w3s3 = LpVariable('w3s3', lowBound=0)
w3s4 = LpVariable('w3s2', lowBound=0)
w3s5 = LpVariable('w3s3', lowBound=0)

w4s1 = LpVariable('w4s1', lowBound=0)
w4s2 = LpVariable('w4s2', lowBound=0)
w4s3 = LpVariable('w4s3', lowBound=0)
w4s4 = LpVariable('w4s2', lowBound=0)
w4s5 = LpVariable('w4s3', lowBound=0)

w5s1 = LpVariable('w5s1', lowBound=0)
w5s2 = LpVariable('w5s2', lowBound=0)
w5s3 = LpVariable('w5s3', lowBound=0)
w5s4 = LpVariable('w5s2', lowBound=0)

#целевая функция
problem += 1000*w1t1 + 12*w1s2 + 10*w1s3 + 19*w1s4 + 8*w1s5 +


#огранич по складам
problem += w1s1 + w1s2 + w1s3 == 180
problem += w2s1 + w2s2 + w2s3 == 220

#ограничения по ТЦ
problem += w1s1 + w2s1 == 110
problem += w1s2 + w2s2 == 150
problem += w1s3 + w2s3 == 140

problem.solve()

print(value(problem.objective))